In [1]:
import fn_module

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yukti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = fn_module.read_file("CSE508_Winter2023_Dataset_Processed_new")
print("File count = ", len(data))

keys = list(data.keys())

s = ""
df = []
for key in data:
    df.append(data[key])


File count =  1400


In [98]:
def create_inverted_index(documents):
    inverted_index = {}
    
    
    for doc_id, document in enumerate(documents):
        string1 = document.split()
        for word in range(len(string1)-1):
            bi = string1[word] + " " + string1[word+1]
            if bi not in inverted_index:
                inverted_index[bi] = []
            inverted_index[bi].append(doc_id)
    
    return inverted_index

inverted_index = create_inverted_index(df)


In [67]:
import pickle
from collections import defaultdict

inverted_index = create_inverted_index(df)

def save_inverted_index(inverted_index, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(inverted_index, f)
        
def load_inverted_index(file_path):
    with open(file_path, 'rb') as f:
        inverted_index = pickle.load(f)
        
    return inverted_index

In [99]:

def create_positional_index(documents):
    pos_index = {}
    for no_1, doc in enumerate(documents):
        conv_lower = doc.lower()
        for no_2, token_no in enumerate(conv_lower.split()):
            if token_no not in pos_index:
                pos_index[token_no] = {}
            if no_1 not in pos_index[token_no]:
                pos_index[token_no][no_1] = []
            pos_index[token_no][no_1].append(no_2)
    return pos_index

pos_index = create_positional_index(df)


In [100]:
import pickle 
def save_positional_index(pos_index, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(pos_index, f)
        
def load_positional_index(file_path):
    with open(file_path, 'rb') as f:
        pos_index = pickle.load(f)
    return pos_index

In [101]:
def and_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    count_comparisons = 0
    if(s1 is not None) and (s2 is not None):
        s_1 = sorted(s1)
        s_2 = sorted(s2)
        len_1 = len(s_1)
        len_2 = len(s_2)
        while(ptr_1 < len_1 and ptr_2 < len_2):
            if s_1[ptr_1]==s_2[ptr_2]:
                ptr_2 = ptr_2 + 1
                final_set.add(s_1[ptr_1])
                ptr_1 = ptr_1 + 1
            elif s_1[ptr_1] > s_2[ptr_2]:
                ptr_2 = ptr_2 + 1
            else:
                ptr_1 = ptr_1 + 1
            count_comparisons += 1
    return final_set, count_comparisons



In [102]:
def get_list(inverted_index , word):
    lists = inverted_index.get(word)
    lists_2 = []
    if lists is not None:
        return lists
    else:
        return lists_2
    return inverted_index.get(word)

In [103]:
def retrieve_documents_bigram(bi_words, inverted_index, log = False):
    count = 0
    l1 = []

    if len(bi_words)>0:
        l1 = get_list(inverted_index,bi_words[0])
        if log:
            print("L1 :",l1)
        for i in range(1,len(bi_words)):
            l2 = get_list(inverted_index,bi_words[i])
            
            l1, count = and_operation(l1,l2)

            if log:
                print("L"+str(i+1),":",l2)
                print("Final Output after Performing AND :",l1)
                
    return l1,count

In [104]:
def retrieve_documents_positional(positional_index, terms, log = False):
    if len(terms) == 0:
        return []
    if terms[0] not in positional_index:
        s1 = set()
    else:
        s1 = set(positional_index[terms[0]].keys())
    if log:
        print("L1 =",list(s1))
    
    for term in terms:
        if term in positional_index:
            s1 = set.intersection(s1,set(positional_index[term].keys()))
            if log:
                print("L2 =",set(positional_index[term].keys()))
                print("List after AND Operation =",list(s1))
    return list(s1)

In [22]:

t = int(input())
for _ in range(t):
    sentence = input()
    trans_sentence = fn_module.preprocess(sentence)
    print("Processed sequence : ",trans_sentence)
    
    bi_words = []
    for word in range(len(trans_sentence)-1):
        string = trans_sentence[word] +" "+ trans_sentence[word+1]
        bi_words.append(string)
        
    print("bi_word sequence : ",bi_words)

    # l,count = retrieve_documents_bigram(bi_words, inverted_index, True)
    l,count = retrieve_documents_bigram(bi_words, inverted_index)
    
    print("Number of documents retrieved for query "+str(_+1)+" using bigram inverted index :",len(l))
  
    print("Names of the documents retrieved for query "+str(_+1)+" using bigram inverted index :")
    
    if len(l)==0:
        print("NO DOCUMENT FOUND!!")
    else:
        doc_lst = []
        for val in l:
            doc_lst.append(keys[val])
        print(doc_lst)
    # print("Number of comparisons required for query "+str(_+1)+" using bigram inverted index :",count)


    # pos_lst = retrieve_documents_positional(pos_index,trans_sentence, True)
    pos_lst = retrieve_documents_positional(pos_index,trans_sentence)

    print("Number of documents retrieved for query "+str(_+1)+" using positional index :",len(pos_lst))
  
    print("Names of the documents retrieved for query "+str(_+1)+" using positional index :")
    
    if len(pos_lst)==0:
        print("NO DOCUMENT FOUND!!")
    else:
        doc_lst = []
        for val in pos_lst:
            doc_lst.append(keys[val])
        print(doc_lst)
    print("-"*150)
    

    

Processed sequence :  ['bluntnosed', 'slender', 'larger']
bi_word sequence :  ['bluntnosed slender', 'slender larger']
L1 : [24, 24, 24, 159, 455]
L2 : []
Final Output after Performing AND : set()
Number of documents retrieved for query 1 using bigram inverted index : 0
Names of the documents retrieved for query 1 using bigram inverted index :
NO DOCUMENT FOUND!!
L1 = [1280, 138, 24, 543, 159, 293, 422, 1081, 575, 455, 1355, 975, 592, 596, 472, 348, 353, 100, 1252, 999, 1005, 1392, 369]
L2 = {1280, 138, 24, 543, 159, 293, 422, 1081, 575, 455, 1355, 975, 592, 596, 472, 348, 353, 100, 1252, 999, 1005, 1392, 369}
List after AND Operation = [1280, 138, 24, 543, 159, 293, 422, 1081, 575, 455, 1355, 975, 592, 596, 472, 348, 353, 100, 1252, 999, 1005, 1392, 369]
L2 = {512, 519, 527, 24, 26, 540, 544, 43, 555, 556, 568, 59, 571, 62, 576, 68, 598, 599, 600, 1111, 604, 624, 122, 145, 146, 159, 1185, 682, 1196, 687, 1200, 1209, 1212, 190, 191, 204, 216, 1247, 738, 228, 229, 745, 234, 1258, 751, 2